#### Schema

_Pydantic_ allows auto creation of JSON Schemas from models.

In [1]:
from enum import Enum
from pydantic import BaseModel, Field

In [2]:
class FooBar(BaseModel):
    count: int
    size: float = None

In [3]:
class Gender(str, Enum):
    male = "male"
    female = "female"
    other = "other"
    not_given = "not_given"

In [4]:
class MainModel(BaseModel):
    """
    This is the description of the main model
    """

    foo_bar: FooBar = Field(...)
    gender: Gender = Field(None, alias="Gender")
    snap: int = Field(
        42,
        title="The Snap",
        description="this is the value of snap",
        gt=30,
        lt=50,
    )

    class Config:
        title = "Main"

In [5]:
print(MainModel.schema_json(indent=2))

{
  "title": "Main",
  "description": "This is the description of the main model",
  "type": "object",
  "properties": {
    "foo_bar": {
      "$ref": "#/definitions/FooBar"
    },
    "Gender": {
      "$ref": "#/definitions/Gender"
    },
    "snap": {
      "title": "The Snap",
      "description": "this is the value of snap",
      "default": 42,
      "exclusiveMinimum": 30,
      "exclusiveMaximum": 50,
      "type": "integer"
    }
  },
  "required": [
    "foo_bar"
  ],
  "definitions": {
    "FooBar": {
      "title": "FooBar",
      "type": "object",
      "properties": {
        "count": {
          "title": "Count",
          "type": "integer"
        },
        "size": {
          "title": "Size",
          "type": "number"
        }
      },
      "required": [
        "count"
      ]
    },
    "Gender": {
      "title": "Gender",
      "description": "An enumeration.",
      "enum": [
        "male",
        "female",
        "other",
        "not_given"
      ],
     

The generated schemas are compliant with the specifications: `JSON Schema Core`, `JSON Schema Validation` and `OpenAPI`.

`BaseModel.schema` will return a dict of the schema, while `BaseModel.schema_json` will return a JSON string representation of that dict.

Sub-models used are added to the `definitions` JSON attribute and referenced, as per the spec.

All sub-models' (and their sub-models') schemas are put directly in a top-level `definitions` JSON key for easy re-use and reference.

"Sub-models" with modifications (via the `Field` class) like a custom title, description or default value, are recursively included instead of referenced.

The `description` for models is taken from either the docstring of the class or the argument `description` to the `Field` class.

The schema is generated by default using aliases as keys, but it can be generated using model property names instead by calling `MainModel.schema/schema_json(by_alias=False)`.

The format of `$ref`s (`"#/definitions/FooBar"` above) can be altered by calling `schema()` or `schema_json()` with the `ref_template` keyword argument, e.g. `ApplePie.schema(ref_template='/schemas/{model}.json#/')`, here `{model}` will be replaced with the model naming using `str.format()`.